In [ ]:
import numpy as np
import imutils
import cv2 as cv
from matplotlib import pyplot as plt
from skimage import io
import os
from os.path import expanduser
HOME = expanduser("~")
SCALED =  'programming/dk43/scaled'
INVERTED = 'programming/dk43/inverted'
ROTATED = 'programming/dk43/rotated'
NORMALIZED = 'programming/dk43/normalized'
MASKED = 'programming/dk43/masked'

In [ ]:
def is_barcode(rect):
    x,y,w,h = rect
    area = w * h
    result = (x > 10 and y > 10 and w > 10 and h > 40 and area > 1000)
    if result:
        print ('rectangle size: x=%s y=%s w=%s h=%s' %(x,y,w,h))
    return result

In [ ]:
def create_mask(original):
    copied = np.copy(original)
    #original = cv2.normalize(original, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    #img_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    #img_blur = cv2.GaussianBlur(img_gray,(5,5),0)
    #img_bgr = cv2.cvtColor(img_blur,cv2.COLOR_GRAY2BGR)
    
    img_hsv = cv.cvtColor(original, cv.COLOR_BGR2HSV)

    lowHue = 0
    lowSat = 0
    lowVal = 2
    highHue = 255
    highSat = 55
    highVal = 250

    # creation of mask
    colorLow = np.array([lowHue,lowSat,lowVal])
    colorHigh = np.array([highHue,highSat,highVal])
    img_mask = cv.inRange(img_hsv, colorLow, colorHigh)
    # find and draw contours
    #find
    contours, hierarchy = cv.findContours(img_mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    #find largest contour
    contour_sizes = [(cv.contourArea(contour), contour) for contour in contours]
    biggest_contour = [max(contour_sizes, key=lambda x: x[0])[1]]
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    #contours = contours
    color = (255,100,0)
    #contour_and_sizes = [(cv.contourArea(contour), contour) for contour in contours]
    #biggest_contours = sorted(contour_and_sizes, key = lambda x: x[0], reverse=True)[:2]

    for i, c in enumerate(contours):
        contours_poly[i] = cv.approxPolyDP(c, 3, True)
        boundRect[i] = cv.boundingRect(contours_poly[i])
        if is_barcode(boundRect[i]):
            centers[i], radius[i] = cv.minEnclosingCircle(contours_poly[i])
        
            cv.rectangle(copied, (int(boundRect[i][0]), int(boundRect[i][1])), \
              (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
        
        
    #cv.fillPoly(copied, [biggest_contour], [255,155,0])
    
    return copied

In [ ]:
def create_rects(original):
    copied = original.copy()
    copied = np.copy(original)
    img_hsv = cv.cvtColor(original, cv.COLOR_BGR2HSV)

    lowHue = 0
    lowSat = 0
    lowVal = 2
    highHue = 255
    highSat = 55
    highVal = 250

    # creation of mask
    colorLow = np.array([lowHue,lowSat,lowVal])
    colorHigh = np.array([highHue,highSat,highVal])
    img_mask = cv.inRange(img_hsv, colorLow, colorHigh)
    # find and draw contours
    contours, hierarchy = cv.findContours(img_mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    #find largest contours
    contour_and_sizes = [(cv.contourArea(contour), contour) for contour in contours]
    biggest_contours = sorted(contour_and_sizes, key = lambda x: x[0], reverse=True)[0]
    for i, c in enumerate(biggest_contours):
        contours_poly[i] = cv.approxPolyDP(c, 3, True)
        boundRect[i] = cv.boundingRect(contours_poly[i])
        centers[i], radius[i] = cv.minEnclosingCircle(contours_poly[i])
        
        cv.rectangle(copied, (int(boundRect[i][0]), int(boundRect[i][1])), \
          (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
        
        
        #cv.fillPoly(copied, [biggest_contour], [255,155,0])
    
    return copied


In [ ]:
#file = 'DK43_slide020_2020_01_23_798515_C0.tif'
#file = 'DK43_slide001_2020_01_27_808119_C0.tif'
file = 'DK43_slide052_2020_01_23_801622_C0.tif'

filename = os.path.join(HOME, NORMALIZED, file)
original = cv.imread(filename)
tissue = create_mask(original)

In [ ]:
# display images
fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(28, 20))
ax[0].set_title('Original')
ax[0].imshow(original)
ax[1].set_title('Mask')
ax[1].imshow(tissue)